<h3>1. Import and set up Conn <h3>

In [1]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import math
connection_uri = (
    "mssql+pyodbc://TEDPHO0134\TANDT_DP/DP_Tan?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri)

In [2]:
def agent_structure(cutoff):
    query_statement = f"select * from AGENT_INFO where CUTOFF = {cutoff}"
    agent_info = pd.read_sql(query_statement, conn)
    #Make Parent table
    parent = agent_info[['Parent_Supervisor_Code', 'Parent_Supervisor_Name', 'Agent_Number', 'Agent_Name', 'Grade', 'CUTOFF']]
    parent = parent[parent['Parent_Supervisor_Code'].notnull() & parent['Parent_Supervisor_Name'].notnull()]
    parent.insert(0, "Mark_Level", "B", True) #ADD a new columm named Mark_Level
    parent = parent.rename(columns={"Parent_Supervisor_Code": "HigherCode", "Parent_Supervisor_Name":"HigherName"})
    #Make Sup table
    sup = agent_info[['Supervisor_Code', 'Supervisor_Name', 'Agent_Number', 'Agent_Name', 'Grade', 'CUTOFF']]
    sup = sup[sup['Supervisor_Code'].notnull() & sup['Supervisor_Name'].notnull()]
    sup.insert(0, "Mark_Level", "A", True)
    sup = sup.rename(columns={"Supervisor_Code": "HigherCode", "Supervisor_Name":"HigherName"})
    parent_sup = pd.concat([parent, sup]) #this is like UNION in SQL
    parent_sup['rank'] = parent_sup.groupby('Agent_Number')['Mark_Level'].rank(method='dense', ascending=True)
    parent_sup = parent_sup[parent_sup['rank'] == 1]
    parent_sup = parent_sup[['HigherCode','HigherName','Agent_Number','Agent_Name','Grade', 'CUTOFF']]
    parent_sup.insert(2, "HigherGrade", parent_sup['HigherName'].str[:2], True)
    parent_sup.insert(2, "Higher Name", parent_sup['HigherName'].str.split('-').str[1] , True)
    parent_sup = parent_sup[['CUTOFF','Agent_Number','Agent_Name','Grade', 'HigherCode','Higher Name','HigherGrade']].rename(columns={"Higher Name":"HigherName"})
    parent_sup_short = parent_sup[['HigherCode','HigherName','HigherGrade','Agent_Number']]
    level_1 = parent_sup.rename(columns={
                                    "Agent_Number":"AgentNumber", "Agent_Name":"AgentName", "Grade":"AgentGrade",
                                    'HigherCode':"L1Code", "HigherName":"L1Name", "HigherGrade":"L1Grade"
                                    })
    level_2 = pd.merge(level_1, parent_sup_short, left_on = 'L1Code', right_on = 'Agent_Number', how = 'left')
    #LEFT JOIN 
    level_2 = level_2.loc[:,level_2.columns != 'Agent_Number'].rename(columns={'HigherCode':"L2Code",
                                                        "HigherName":"L2Name", "HigherGrade":"L2Grade", "CUTOFF_x": "CUTOFF"})
    #level_2.loc[:,level_2.columns != 'Agent_Number'] => SELECT ALL COLUMNS, EXCEPT Agent_Number
    level_3 = pd.merge(level_2, parent_sup_short, left_on = 'L2Code', right_on = 'Agent_Number', how = 'left')
    level_3 = level_3.loc[:,level_3.columns != 'Agent_Number'].rename(columns={'HigherCode':"L3Code",
                                                        "HigherName":"L3Name", "HigherGrade":"L3Grade", "CUTOFF_x": "CUTOFF"})
    level_4 = pd.merge(level_3, parent_sup_short, left_on = 'L3Code', right_on = 'Agent_Number', how = 'left')
    level_4 = level_4.loc[:,level_4.columns != 'Agent_Number'].rename(columns={'HigherCode':"L4Code",
                                                        "HigherName":"L4Name", "HigherGrade":"L4Grade", "CUTOFF_x": "CUTOFF"})
    level_5 = pd.merge(level_4, parent_sup_short, left_on = 'L4Code', right_on = 'Agent_Number', how = 'left')
    level_5 = level_5.loc[:,level_5.columns != 'Agent_Number'].rename(columns={'HigherCode':"L5Code",
                                                        "HigherName":"L5Name", "HigherGrade":"L5Grade", "CUTOFF_x": "CUTOFF"})
    level_6 = pd.merge(level_5, parent_sup_short, left_on = 'L5Code', right_on = 'Agent_Number', how = 'left')
    level_6 = level_6.loc[:,level_6.columns != 'Agent_Number'].rename(columns={'HigherCode':"L6Code",
                                                        "HigherName":"L6Name", "HigherGrade":"L6Grade", "CUTOFF_x": "CUTOFF"})
    level_7 = pd.merge(level_6, parent_sup_short, left_on = 'L6Code', right_on = 'Agent_Number', how = 'left')
    level_7 = level_7.loc[:,level_7.columns != 'Agent_Number'].rename(columns={'HigherCode':"L7Code",
                                                        "HigherName":"L7Name", "HigherGrade":"L7Grade", "CUTOFF_x": "CUTOFF"})
    level_8 = pd.merge(level_7, parent_sup_short, left_on = 'L7Code', right_on = 'Agent_Number', how = 'left')
    level_8 = level_8.loc[:,level_8.columns != 'Agent_Number'].rename(columns={'HigherCode':"L8Code",
                                                        "HigherName":"L8Name", "HigherGrade":"L8Grade", "CUTOFF_x": "CUTOFF"})
    level_9 = pd.merge(level_8, parent_sup_short, left_on = 'L8Code', right_on = 'Agent_Number', how = 'left')
    level_9 = level_9.loc[:,level_9.columns != 'Agent_Number'].rename(columns={'HigherCode':"L9Code",
                                                        "HigherName":"L9Name", "HigherGrade":"L9Grade", "CUTOFF_x": "CUTOFF"})
    level_10 = pd.merge(level_9, parent_sup_short, left_on = 'L9Code', right_on = 'Agent_Number', how = 'left')
    level_10 = level_10.loc[:,level_10.columns != 'Agent_Number'].rename(columns={'HigherCode':"L10Code",
                                                        "HigherName":"L10Name", "HigherGrade":"L10Grade", "CUTOFF_x": "CUTOFF"})
    level_11 = pd.merge(level_10, parent_sup_short, left_on = 'L10Code', right_on = 'Agent_Number', how = 'left')
    level_11 = level_11.loc[:,level_11.columns != 'Agent_Number'].rename(columns={'HigherCode':"L11Code",
                                                        "HigherName":"L11Name", "HigherGrade":"L11Grade", "CUTOFF_x": "CUTOFF"})
    level_12 = pd.merge(level_11, parent_sup_short, left_on = 'L11Code', right_on = 'Agent_Number', how = 'left')
    level_12 = level_12.loc[:,level_12.columns != 'Agent_Number'].rename(columns={'HigherCode':"L12Code",
                                                        "HigherName":"L12Name", "HigherGrade":"L12Grade", "CUTOFF_x": "CUTOFF"})
    level_13 = pd.merge(level_12, parent_sup_short, left_on = 'L12Code', right_on = 'Agent_Number', how = 'left')
    level_13 = level_13.loc[:,level_13.columns != 'Agent_Number'].rename(columns={'HigherCode':"L13Code",
                                                        "HigherName":"L13Name", "HigherGrade":"L13Grade", "CUTOFF_x": "CUTOFF"})
    final_tree = level_13
    #Lấy ra FM
    final_tree.loc[final_tree['AgentGrade'] == "FM", 'FM_Code'] = final_tree['AgentNumber']
    final_tree.loc[final_tree['AgentGrade'] == "FM", 'FM_Name'] = final_tree['AgentName']
    #Lấy ra DM
    final_tree.loc[final_tree['AgentGrade'] == "DM", 'DM_Code'] = final_tree['AgentNumber']
    final_tree.loc[final_tree['AgentGrade'] == "DM", 'DM_Name'] = final_tree['AgentName']
    #Lấy ra RM
    final_tree.loc[final_tree['AgentGrade'] == "RM", 'RM_Code'] = final_tree['AgentNumber']
    final_tree.loc[final_tree['AgentGrade'] == "RM", 'RM_Name'] = final_tree['AgentName']
    #Lấy ra GM
    final_tree.loc[final_tree['AgentGrade'] == "GM", 'GM_Code'] = final_tree['AgentNumber']
    final_tree.loc[final_tree['AgentGrade'] == "GM", 'GM_Name'] = final_tree['AgentName']
    #List code
    code_lst = [ ("L" +str(i)+"Code") for i in range(13,0,-1)]
    name_lst = [ ("L" +str(i)+"Name") for i in range(13,0,-1)]
    grade_lst = [ ("L" +str(i)+"Grade") for i in range(13,0,-1)]
    for code,name, grade in zip(code_lst, name_lst, grade_lst):
        final_tree.loc[(final_tree[grade] == "FM") & (final_tree[grade] != final_tree['AgentGrade']), "FM_Code"] = final_tree[code]
        final_tree.loc[(final_tree[grade] == "FM") & (final_tree[grade] != final_tree['AgentGrade']), "FM_Name"] = final_tree[name]
        #
        final_tree.loc[(final_tree[grade] == "DM") & (final_tree[grade] != final_tree['AgentGrade']), "DM_Code"] = final_tree[code]
        final_tree.loc[(final_tree[grade] == "DM") & (final_tree[grade] != final_tree['AgentGrade']), "DM_Name"] = final_tree[name]
        #
        final_tree.loc[(final_tree[grade] == "RM") & (final_tree[grade] != final_tree['AgentGrade']), "RM_Code"] = final_tree[code]
        final_tree.loc[(final_tree[grade] == "RM") & (final_tree[grade] != final_tree['AgentGrade']), "RM_Name"] = final_tree[name]
        #
        final_tree.loc[(final_tree[grade] == "GM") & (final_tree[grade] != final_tree['AgentGrade']), "GM_Code"] = final_tree[code]
        final_tree.loc[(final_tree[grade] == "GM") & (final_tree[grade] != final_tree['AgentGrade']), "GM_Name"] = final_tree[name]
    change_grade = ['AM', 'IS', 'IG', 'ID', 'EA', 'PA']
    for grad in change_grade:
        final_tree['AgentGrade'] = final_tree['AgentGrade'].replace(grad, "FC")
    #Lấy ra FC Direct
    Agent_List = final_tree[['GM_Code', 'GM_Name', 'RM_Code', 'RM_Name', 'DM_Code', 'DM_Name', 'FM_Code', 'FM_Name',
        'CUTOFF', 'AgentNumber', 'AgentName', 'AgentGrade']]
    FC_list_code = ['GM_Code', 'RM_Code', 'DM_Code', 'FM_Code']
    FC_list_name = ['GM_Name', 'RM_Name', 'DM_Name', 'FM_Name']
    for code, name in zip(FC_list_code, FC_list_name):
        Agent_List.loc[((Agent_List['AgentGrade']=="FC") | (Agent_List[code] == Agent_List['AgentNumber'])) & (Agent_List[code].notnull()), "DirFC_Code"] = Agent_List[code]
        Agent_List.loc[((Agent_List['AgentGrade']=="FC") | (Agent_List[code] == Agent_List['AgentNumber'])) & (Agent_List[name].notnull()), "DirFC_Name"] = Agent_List[name]
    #Lấy ra danh sách Indirect
    FM_Indirect = parent_sup[(parent_sup['Grade']== "FM") & (parent_sup['HigherGrade']=="FM")]
    DM_Indirect = parent_sup[(parent_sup['Grade']== "DM") & (parent_sup['HigherGrade']=="DM")]
    RM_Indirect = parent_sup[(parent_sup['Grade']== "RM") & (parent_sup['HigherGrade']=="RM")]
    GM_Indirect = parent_sup[(parent_sup['Grade']== "GM") & (parent_sup['HigherGrade']=="GM")]
    #FM Ind
    Agent_With_Ind = pd.merge(Agent_List, FM_Indirect, left_on = "FM_Code", right_on = "Agent_Number", how = 'left')[[
    'GM_Code', 'GM_Name', 'RM_Code', 'RM_Name', 'DM_Code', 'DM_Name', 'HigherCode', 'HigherName', 'FM_Code', 'FM_Name', 
    'DirFC_Code','DirFC_Name','AgentNumber', 'AgentName','AgentGrade']].rename(columns={"HigherCode":"FM_IndCode", "HigherName":"FM_IndName"})  
    #DM Ind
    Agent_With_Ind = pd.merge(Agent_With_Ind, DM_Indirect, left_on = "DM_Code", right_on = "Agent_Number", how = 'left')[[
    'GM_Code', 'GM_Name', 'RM_Code', 'RM_Name','HigherCode', 'HigherName' ,'DM_Code', 'DM_Name', 
    'FM_IndCode','FM_IndName','FM_Code', 'FM_Name', 
    'DirFC_Code','DirFC_Name','AgentNumber', 'AgentName','AgentGrade']].rename(columns={"HigherCode":"DM_IndCode", "HigherName":"DM_IndName"})   
    #RM Ind
    Agent_With_Ind = pd.merge(Agent_With_Ind, RM_Indirect, left_on = "RM_Code", right_on = "Agent_Number", how = 'left')[[
    'GM_Code', 'GM_Name', 'HigherCode', 'HigherName', 'RM_Code', 'RM_Name',
    'DM_IndCode','DM_IndName','DM_Code', 'DM_Name', 
    'FM_IndCode','FM_IndName','FM_Code', 'FM_Name', 
    'DirFC_Code','DirFC_Name','AgentNumber', 'AgentName','AgentGrade']].rename(columns={"HigherCode":"RM_IndCode", "HigherName":"RM_IndName"}) 
    #GM Ind
    Agent_With_Ind = pd.merge(Agent_With_Ind, GM_Indirect, left_on = "GM_Code", right_on = "Agent_Number", how = 'left')[[
    'HigherCode', 'HigherName','GM_Code', 'GM_Name',  
    'RM_IndCode','RM_IndName','RM_Code', 'RM_Name',
    'DM_IndCode','DM_IndName','DM_Code', 'DM_Name', 
    'FM_IndCode','FM_IndName','FM_Code', 'FM_Name', 
    'DirFC_Code','DirFC_Name','AgentNumber', 'AgentName','AgentGrade']].rename(columns={"HigherCode":"GM_IndCode", "HigherName":"GM_IndName"})
    #Lấy ra WholeGroup
    ind_Code = ['GM_IndCode', 'RM_IndCode', 'DM_IndCode', 'FM_IndCode']
    dir_Code = ['GM_Code', 'RM_Code', 'DM_Code', 'FM_Code']
    whole_Code = ['GM_Whole_Code', 'RM_Whole_Code', 'DM_Whole_Code', 'FM_Whole_Code']
    ind_Name = ['GM_IndName', 'RM_IndName', 'DM_IndName', 'FM_IndName']
    dir_Name = ['GM_Name', 'RM_Name', 'DM_Name', 'FM_Name']
    whole_Name = ['GM_Whole_Name', 'RM_Whole_Name', 'DM_Whole_Name', 'FM_Whole_Name']
    for ind_C, dir_C, whole_C in zip (ind_Code, dir_Code, whole_Code):
        Agent_With_Ind.loc[(Agent_With_Ind[ind_C].notnull()) & (Agent_With_Ind[dir_C].notnull()), whole_C] = Agent_With_Ind[ind_C]
        Agent_With_Ind.loc[(Agent_With_Ind[ind_C].isnull()), whole_C] = Agent_With_Ind[dir_C]
        #
    for ind_C, dir_C, whole_C in zip (ind_Name, dir_Name, whole_Name):
        Agent_With_Ind.loc[(Agent_With_Ind[ind_C].notnull()) & (Agent_With_Ind[dir_C].notnull()), whole_C] = Agent_With_Ind[ind_C]
        Agent_With_Ind.loc[(Agent_With_Ind[ind_C].isnull()), whole_C] = Agent_With_Ind[dir_C]
    
    #Lấy ra Agent trực tiếp của AD
    agent_info_test = agent_info
    agent_info_test = agent_info_test.replace(r'^\s*$', np.nan, regex=True)
    agent_info_test.replace(np.nan, 0, inplace = True )
    #
    Agent_AD = agent_info_test[(agent_info_test['Parent_Supervisor_Code'] == 0) & (agent_info_test['Supervisor_Code'] == 0) ]
    Agent_AD.replace(0,np.nan, inplace= True) #replace from 0 => NaN để tránh trùng lặp
    clone_columns = {'GM_IndCode': np.nan,  'GM_IndName': np.nan,  'RM_IndCode': np.nan, 
       'RM_IndName': np.nan,  'DM_IndCode': np.nan,  'DM_IndName': np.nan, 
       'FM_IndCode': np.nan,  'FM_IndName': np.nan
    }
    #
    Agent_AD = Agent_AD[['Agent_Number', 'Agent_Name', 'Grade']].assign(**clone_columns)
    Agent_AD = Agent_AD.rename(columns= {'Agent_Number': 'AgentNumber', 'Agent_Name':'AgentName', 'Grade':'AgentGrade'})
    #Lấy ra FM
    Agent_AD.loc[Agent_AD['AgentGrade'] == "FM", 'FM_Code'] = Agent_AD['AgentNumber']
    Agent_AD.loc[Agent_AD['AgentGrade'] == "FM", 'FM_Name'] = Agent_AD['AgentName']
    #Lấy ra DM
    Agent_AD.loc[Agent_AD['AgentGrade'] == "DM", 'DM_Code'] = Agent_AD['AgentNumber']
    Agent_AD.loc[Agent_AD['AgentGrade'] == "DM", 'DM_Name'] = Agent_AD['AgentName']
    #Lấy ra RM
    Agent_AD.loc[Agent_AD['AgentGrade'] == "RM", 'RM_Code'] = Agent_AD['AgentNumber']
    Agent_AD.loc[Agent_AD['AgentGrade'] == "RM", 'RM_Name'] = Agent_AD['AgentName']
    #Lấy ra GM
    Agent_AD.loc[Agent_AD['AgentGrade'] == "GM", 'GM_Code'] = Agent_AD['AgentNumber']
    Agent_AD.loc[Agent_AD['AgentGrade'] == "GM", 'GM_Name'] = Agent_AD['AgentName']
    #Lấy ra Whole:
    for dir_C, whole_C in zip (dir_Code, whole_Code):
        Agent_AD[whole_C] = Agent_AD[dir_C]
        #
    for dir_C, whole_C in zip (dir_Name, whole_Name):
        Agent_AD[whole_C] = Agent_AD[dir_C]
    # Lấy ra Direct FC:
    Agent_AD.loc[(Agent_AD['AgentGrade'] == "GM") | (Agent_AD['AgentGrade'] == "RM") |
                 (Agent_AD['AgentGrade'] == "DM") | (Agent_AD['AgentGrade'] == "FM") 
                 , 'DirFC_Code'] = Agent_AD['AgentNumber']
    Agent_AD.loc[(Agent_AD['AgentGrade'] == "GM") | (Agent_AD['AgentGrade'] == "RM") |
                 (Agent_AD['AgentGrade'] == "DM") | (Agent_AD['AgentGrade'] == "FM") 
                 , 'DirFC_Name'] = Agent_AD['AgentName']
    Agent_AD = Agent_AD[['GM_IndCode', 'GM_IndName',
       'GM_Code', 'GM_Name', 'RM_IndCode', 'RM_IndName', 'RM_Code', 'RM_Name',
       'DM_IndCode', 'DM_IndName', 'DM_Code', 'DM_Name', 'FM_IndCode',
       'FM_IndName', 'FM_Code', 'FM_Name', 'DirFC_Code', 'DirFC_Name', 'AgentNumber', 'AgentName', 'AgentGrade',
        'GM_Whole_Code', 'RM_Whole_Code', 'DM_Whole_Code', 'FM_Whole_Code', 'GM_Whole_Name', 'RM_Whole_Name',
        'DM_Whole_Name','FM_Whole_Name']]
    #Lấy full list Agent:
    Full_Agent = pd.concat([Agent_With_Ind, Agent_AD])
    #Lấy thông tin còn thiếu 
    Full_Agent_Info = pd.merge(Full_Agent, agent_info, left_on ='AgentNumber', right_on='Agent_Number', how = 'inner')
    Full_Agent_Info = Full_Agent_Info[['Area_Name', 'Sales_Unit_Code', 'Sales_Unit', 
        'GM_Whole_Code', 'GM_Whole_Name','GM_IndCode', 'GM_IndName', 'GM_Code', 'GM_Name', 
        'RM_Whole_Code','RM_Whole_Name','RM_IndCode', 'RM_IndName', 'RM_Code', 'RM_Name', 
        'DM_Whole_Code', 'DM_Whole_Name','DM_IndCode', 'DM_IndName','DM_Code', 'DM_Name', 
        'FM_Whole_Code','FM_Whole_Name','FM_IndCode', 'FM_IndName', 'FM_Code', 'FM_Name',
        'DirFC_Code', 'DirFC_Name', 'AgentNumber', 'AgentName', 'AgentGrade', 
        'Gender', 'Agent_Status', 'ID_Card', 'Hand_Phone', 'Date_Appointed', 'Terminated_date', 'Promote_Date',
        'Demote_Date', 'TERRITORY', 'Leader', 'Parent_Supervisor_Code','Supervisor_Code','CUTOFF']]
    #Loại bỏ khoảng trắng
    col_full = Full_Agent_Info.columns.to_list()
    try:
        for col in col_full:
            Full_Agent_Info[col] = Full_Agent_Info[col].str.strip()
    except:
        pass    
    return Full_Agent_Info

<h3> Đẩy vào Database <h3>

In [16]:
#Tạo bảng
#df.to_sql("BSP_AGENT_STRUCTURE", conn, if_exists='replace', index=False)

2

In [23]:
df.to_csv("test1.csv", encoding = 'utf-8-sig')

In [3]:
df = agent_structure("20240229")

C:\Users\vagy1255\AppData\Local\Temp\ipykernel_10096\3116100759.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Agent_List.loc[((Agent_List['AgentGrade']=="FC") | (Agent_List[code] == Agent_List['AgentNumber'])) & (Agent_List[code].notnull()), "DirFC_Code"] = Agent_List[code]
C:\Users\vagy1255\AppData\Local\Temp\ipykernel_10096\3116100759.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Agent_List.loc[((Agent_List['AgentGrade']=="FC") | (Agent_List[code] == Agent_List['AgentNumber'])) & (Agent_Li

In [4]:
df2 = df.fillna(value=0) #fillna để tránh lỗi trong việc đẩy data vào SQL
#3.Tạo Connection
conn_toSQL = pyo.connect(driver='{ODBC Driver 17 for SQL Server}', host = 'TEDPHO0134\TANDT_DP', database = 'DP_Tan', trusted_connection = 'yes',
                        ansi=True,autocommit=True)
cursor = conn_toSQL.cursor()
# Insert Dataframe into SQL Server:
for index, row in df2.iterrows():
     cursor.execute("INSERT INTO dbo.BSP_AGENT_STRUCTURE (Area_Name, Sales_Unit_Code, Sales_Unit, GM_Whole_Code, GM_Whole_Name, GM_IndCode, GM_IndName, GM_Code, GM_Name, RM_Whole_Code, RM_Whole_Name, RM_IndCode, RM_IndName, RM_Code, RM_Name, DM_Whole_Code, DM_Whole_Name,DM_IndCode,DM_IndName,DM_Code,DM_Name,FM_Whole_Code,FM_Whole_Name,FM_IndCode,FM_IndName,FM_Code,FM_Name,DirFC_Code,DirFC_Name, AgentNumber, AgentName, AgentGrade, Gender, Agent_Status, ID_Card, Hand_Phone,Date_Appointed, Terminated_date, Promote_Date, Demote_Date, TERRITORY, Leader,Parent_Supervisor_Code, Supervisor_Code, CUTOFF) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
, 
row.Area_Name,
row.Sales_Unit_Code,
row.Sales_Unit,
row.GM_Whole_Code,
row.GM_Whole_Name,
row.GM_IndCode,
row.GM_IndName,
row.GM_Code,
row.GM_Name,
row.RM_Whole_Code,
row.RM_Whole_Name,
row.RM_IndCode,
row.RM_IndName,
row.RM_Code,
row.RM_Name,
row.DM_Whole_Code,
row.DM_Whole_Name,
row.DM_IndCode,
row.DM_IndName,
row.DM_Code,
row.DM_Name,
row.FM_Whole_Code,
row.FM_Whole_Name,
row.FM_IndCode,
row.FM_IndName,
row.FM_Code,
row.FM_Name,
row.DirFC_Code,
row.DirFC_Name,
row.AgentNumber,
row.AgentName,
row.AgentGrade,
row.Gender,
row.Agent_Status,
row.ID_Card,
row.Hand_Phone,
row.Date_Appointed,
row.Terminated_date,
row.Promote_Date,
row.Demote_Date,
row.TERRITORY,
row.Leader,
row.Parent_Supervisor_Code,
row.Supervisor_Code,
row.CUTOFF
)
conn_toSQL.commit()
cursor.close()